<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Importing Libraries</h2>
 </div>

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter 
warnings.filterwarnings("ignore")

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Node Class</h2>
 </div>

In [15]:
class Node:
    # value is used as keyword-only argument
    def __init__(self,features=None,threshold=None,left=None,right=None,*,value=None):
        self.features=features
        self.threshold=threshold
        self.left=left
        self.right=right
        self.value=value
     #simple method if current node is leaf node   
    def is_leaf_node(self):
        return self.value is not None

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Decision Tree</h2>
 </div>

In [16]:
class DecisionTreeClassifier:
    def __init__(self,max_depth=100,min_sample_split=2,n_features=None) -> None:
        self.root=None
        self.max_depth=max_depth
        self.min_sample_split=min_sample_split
        self.n_features=n_features

    def fit(self,X,y):
        #if no. of features is None features from X is selected
        self.n_features=X.shape[1] if not self.n_features else min(self.n_features,X.shape[1])
        self.root=self._Tree(X,y)
        
     #recursively builds the tree   
    def _Tree(self,X,y,depth=0):
        try:
            samples,features=X.shape
            n_label=len(np.unique(y))
            
            #stop condition
            if (depth >= self.max_depth or n_label ==1 or samples < self.min_sample_split):
                return Node(value=self._most_common(y))

            #to reduce overfitting the features values are randomly choosen
            feat_idxs=np.random.choice(features,self.n_features,replace=False)
            #getting the best features and thresholds
            best_idxs,best_threshold=self._best_split(X,y,feat_idxs)
            #spliting 
            left_idxs,right_idxs=self._split(X[:,best_idxs],best_threshold)
            #recursively build left part of the tree
            left=self._Tree(X[left_idxs,:],y[left_idxs],depth+1)
            #recursively build right part of the tree
            right=self._Tree(X[right_idxs,:],y[right_idxs],depth+1)
            #return object of class Node
            return Node(best_idxs,best_threshold,left,right)
        except Exception as e:
            print("error",e)

    #greedy way of searching for best feature and threshold
    def _best_split(self,X,y,feat_idxs):
        try:
            best_gain=-1
            split_idxs,split_thres=None,None
            for feat_idx in feat_idxs:
                X_column=X[:,feat_idx]
                thresholds=np.unique(X_column)
                for threshold in thresholds:
                    gain=self._information_gain(X_column,y,threshold)

                    if gain>best_gain:
                        best_gain=gain
                        split_idxs=feat_idx
                        split_thres=threshold
            return split_idxs,split_thres
        except Exception as e:
            print("error",e)

    #used to find best features and best threshold to split the tree 
    #Information Gain = Entropy before splitting - Entropy after splitting
    def _information_gain(self,X_column,y,threshold):
        try:
            parent_entrophy=self._entrophy(y)
            left_idxs,right_idxs=self._split(X_column,threshold)
            #handling empty values
            if len(left_idxs)==0 or len(right_idxs)==0:
                return 0
            n=len(y)
            n_l,n_r=len(left_idxs),len(right_idxs)
            e_l,e_r=self._entrophy(y[left_idxs]),self._entrophy(y[right_idxs])
            child_entrophy=(n_l/n)*e_l +(n_r/n)*e_r
            return parent_entrophy-child_entrophy
        except Exception as e:
            print("error",e)
            
    #split method to split X w.r.t threshold 
    #'<=' and '>' used for numerical values  
    #"==" and "!=" for categorical values 
    def _split(self,X_column,threshold):
        left_idxs=np.where(X_column<=threshold)[0]
        right_idxs=np.where(X_column>threshold)[0]
        return left_idxs,right_idxs
    
    #entrophy sum of -y * log(y) for all classes of y  
    def _entrophy(self,y):
        hist=np.bincount(y)
        ps=hist/len(y)
        return -np.sum(p *np.log(p) for p in ps if p>0)
    
    #recursively called to find most common value of y in that split
    def _most_common(self,y):
        count=Counter(y)
        return count.most_common(1)[0][0]
    
    #traverse untill leaf_node is found
    def _Traverse(self,X,node):
        if node.is_leaf_node():
            return node.value
        if X[node.features]<=node.threshold:
            return self._Traverse(X,node.left)
        return self._Traverse(X,node.right)
        
    def predict(self,X):
        return np.array([self._Traverse(x,self.root) for x in X])


<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Loading Dataset</h2>
 </div>

In [28]:
from sklearn.datasets import load_iris
dataset=load_iris()
X=dataset['data']
Y=dataset['target'] 

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Test Train split</h2>
 </div>

In [32]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=1234)

<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Implementing DecisionTreeClassifier</h2>
 </div>

In [33]:
DR=DecisionTreeClassifier(max_depth=2)
DR.fit(X_train,y_train)
y_pred=DR.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.9333333333333333


<div class="alert alert-box alert-info">
    <h2 style="margin:0px">Changing max depth</h2>
 </div>

In [35]:
for i in range(10):
    DR=DecisionTreeClassifier(max_depth=2)
    DR.fit(X_train,y_train)
    y_pred=DR.predict(X_test)
    from sklearn.metrics import accuracy_score
    print(f"for depth = {i}, score = {accuracy_score(y_test,y_pred)}",)

for depth = 0, score = 0.9333333333333333
for depth = 1, score = 0.9555555555555556
for depth = 2, score = 0.9555555555555556
for depth = 3, score = 0.9555555555555556
for depth = 4, score = 0.9333333333333333
for depth = 5, score = 0.9555555555555556
for depth = 6, score = 0.9333333333333333
for depth = 7, score = 0.9333333333333333
for depth = 8, score = 0.9333333333333333
for depth = 9, score = 0.9555555555555556
